# 07 プランニング: Azure OpenAI 直接利用サンプル (日本語版)

このノートブックは英語版 `07-planning-design/code_samples/07-azure-openai.ipynb` を参考に、Azure OpenAI(互換推論)を用いて
プランニング用の構造化JSONを取得する最小例を示します。

> 注: ここでは GitHub Models 経由のエンドポイントを利用しています。Azure OpenAI 専用デプロイを使う場合は endpoint / key / api_version を調整してください。

### 目的
このノートブックの目的は「自然言語ゴール → LLM 呼び出し → 構造化(JSON)計画 → スキーマ検証 → （必要なら）フォールバック正規化」という最小パイプラインを確認することです。

### 得られる成果物
- Pydantic モデル (Plan / SubTask / AgentEnum) を先に定義し構造化出力を強制する手法
- SystemMessage による JSON キー制約と簡易 few-shot 的サンプル（例付きプロンプト）
- モデルが day / activities のような別形式で返した時のフォールバック正規化例
- 早期検証 (model_validate) による不正フォーマット検出パターン

### セル構成 (学習段階に合わせた責務分離)
| セル | タイトル / 目的 | 主なポイント |
|------|-----------------|--------------|
| 1 | このイントロ | ゴール/範囲/構成 | 
| 2 | インストール & インポート | 依存分離 / 再実行頻度低 | 
| 3 | スキーマ定義 | "出力仕様" を先に固定 | 
| 4 | 環境変数ロード & クライアント | 接続確認 / 再利用関数化 | 
| 5 | プロンプト編集ゾーン | A/B テストしやすい分離 | 
| 6 | 実行 & 正規化 & 検証 & 出力 | 変動が最も多い部分 | 
| 7 | 拡張案 | 次学習ステップ指針 | 

### README との対応
- 「タスク分解」「構造化出力」のコア: 本ノートでスキーマ定義 + 構造化 JSON 生成/検証として再現。
- 「拡張フル例」の基礎: エラーハンドリング（JSON 抽出 + フォールバック正規化）を部分的に包含。
- 「マルチエージェント連携」: 本ノートでは未実装（単一プラン生成のみ）。README で概念理解後、別ノート/拡張で実装可。
- 「反復的プランニング（Re-plan）」: これについては未収録。Plan を保持し再プランプロンプトを追加することで拡張可能。
- 「推奨実行順序」内ステップ1相当: 接続確認 + 最小スキーマ検証を担う位置付け。

> 発展: README の拡張フル例と再プラン節を参考に、(1) 役割説明をより細かくした SystemMessage、(2) 再計画セル、(3) ログ/検証メトリクス収集セルを追加して学習を段階的に進められます。

### 実行手順
1. 必要ライブラリインストールと環境変数(GITHUB_TOKEN など)確認
2. スキーマ定義（列挙型 + Pydantic）
3. プロンプト（System + User）構築 (セル5)
4. 応答(JSON)抽出と例外処理 / フォールバック正規化 (セル6)
5. スキーマ検証 & サブタスク一覧表示 (セル6)

---

In [5]:
# セル2: インストール & インポート (再実行頻度低)
%pip install azure-ai-inference pydantic --quiet
import os, json, re
from typing import List
from enum import Enum
from pydantic import BaseModel
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
print('[INIT] ライブラリ読み込み完了')

Note: you may need to restart the kernel to use updated packages.
[INIT] ライブラリ読み込み完了



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# セル3: スキーマ定義 (LLM 出力スキーマを固定)
# =============================================================
# ★重要: ここで “出力スキーマ” を先に固定することで
#         以降のプロンプト/検証/再計画すべての一貫性を担保する。
#         この段を変えると下流 (正規化/評価) ロジックの想定が崩れる。
#         → 合意された JSON 形式をコード化し揺れを最小化。
# =============================================================
class AgentEnum(str, Enum):
    Flight='flight_booking'
    Hotel='hotel_booking'
    Car='car_rental'
    Activity='activities_booking'
    Destination='destination_info'

class SubTask(BaseModel):
    task_details: str  # ★各サブタスク具体説明
    assigned_agent: AgentEnum  # ★どのエージェントに渡すか (ルーティングキー)

class Plan(BaseModel):  # ★最上位プラン: 解析/表示/再プランの中核データ構造
    main_task: str      #   ゴール要約 (人間/ログ可読性)
    subtasks: List[SubTask]  #   実行単位に分解された手順 (自動化/並列化単位)
    is_greeting: bool   #   挨拶のみ判定 (後段処理簡略)

print('[SCHEMA] AgentEnum / SubTask / Plan 出力スキーマ定義完了')

[SCHEMA] AgentEnum / SubTask / Plan 定義完了 (★Plan スキーマ確定)


In [7]:
# セル4: 環境変数ロード & クライアント生成
# (必要なら python-dotenv 利用可)
TOKEN = os.getenv('GITHUB_TOKEN') or os.getenv('OPENAI_API_KEY')
if not TOKEN:
    raise RuntimeError('GITHUB_TOKEN か OPENAI_API_KEY を設定してください')
ENDPOINT = os.getenv('OPENAI_BASE_URL','https://models.inference.ai.azure.com')
MODEL = os.getenv('OPENAI_MODEL','gpt-4o-mini')
client = ChatCompletionsClient(endpoint=ENDPOINT, credential=AzureKeyCredential(TOKEN))
print('[CLIENT] 初期化成功 endpoint=', ENDPOINT, ' model=', MODEL)

[CLIENT] 初期化成功 endpoint= https://models.inference.ai.azure.com  model= gpt-4o-mini


In [ ]:
# セル5: プロンプト編集ゾーン (A/B テスト用)
# -------------------------------------------------------------
# ★重要: モデルに“何を、どの構造で”返させるかの制御レバー。
#         少しの文言差で構造揺れが発生 → 下流正規化コスト増。
#         ここは実験頻度が高いので他セルから独立。
# -------------------------------------------------------------
goal = '家族4人(子供2人) シンガポール→メルボルン 3日旅行プランを作成'
SYSTEM_PROMPT = (
    'あなたは旅行プラン用プランナーです。必ず以下の JSON 構造のみを出力してください。\n'
    'キー: main_task (文字列), subtasks (配列), is_greeting (true/false)。\n'
    'subtasks 要素: {"task_details": <文字列>, "assigned_agent": < flight_booking | hotel_booking | car_rental | activities_booking | destination_info >}。\n'
    '挨拶のみなら subtasks は空配列, is_greeting=true。\n'
    '例: {"main_task":"旅行計画","subtasks":[{"task_details":"往復航空券を検索","assigned_agent":"flight_booking"}],"is_greeting":false}'
)
print('[PROMPT] goal=', goal)
print('[PROMPT] System 制約キー: main_task / subtasks[] / is_greeting (★構造再確認)')

[PROMPT] goal= 家族4人(子供2人) シンガポール→メルボルン 3日旅行プランを作成
[PROMPT] System 制約キー: main_task / subtasks[] / is_greeting


In [ ]:
# セル6: 実行 & 抽出 & 正規化 & 検証
# === README / セル1 対応マッピング =================================
# このセルはセル1「タスク分解」「構造化出力」の“後半(生成/検証)”を担当。
# - タスク分解: セル3で固定した SubTask / Plan スキーマに沿う形で LLM 応答を取得し
#               subtasks[] に手順分解された要素を格納する工程。
# - 構造化出力: 生テキスト → JSON 抽出 → (必要なら正規化) → Pydantic 検証 のパイプラインで
#               形式保証と揺れ抑制を実現。
# 結合イメージ: [セル3: 仕様(スキーマ)固定] + [セル6: 生成/整形/検証] = README 該当点の再現。
# 品質目的: 出力揺れ可観測性 / 早期Fail Fast / 将来 Re-plan (差分比較) の土台を形成。
# ===================================================================
system = SystemMessage(content=SYSTEM_PROMPT)
user = UserMessage(content=goal)
resp = client.complete(messages=[system, user], model=MODEL, temperature=0.2, max_tokens=800)
text = resp.choices[0].message.content or ''
print('[RAW 頭120]', text[:120].replace('\n',' '), '...')

# ★抽出: 最初の { ... } ブロックを単純に取る (将来はより堅牢なパーサに差し替え可能)
m = re.search(r'\{[\s\S]*\}', text.strip())
json_str = m.group(0) if m else text
print('[PARSE] 抽出候補 先頭90:', json_str[:90], '...')

# ★デコード: 失敗=プロンプトorモデル応答の構造逸脱 → 早期Fail Fast
try:
    data = json.loads(json_str)
except json.JSONDecodeError as e:
    raise ValueError(f'JSON デコード失敗: {e}')

# ★正規化: 想定外(day/activities) 形式→共通 subtasks 形式へ変換
if isinstance(data, dict) and data.get('subtasks') and isinstance(data['subtasks'], list) and data['subtasks'] and 'task_details' not in data['subtasks'][0]:
    print('[WARN] day/activities 構造検出 → 正規化')
    new_subs = []
    for entry in data['subtasks']:
        day_label = entry.get('day')
        acts = entry.get('activities') or []
        for act in acts:
            new_subs.append({'task_details': f'Day {day_label}: {act}', 'assigned_agent':'activities_booking'})
    data['subtasks'] = new_subs
    data.setdefault('is_greeting', False)
    data.setdefault('main_task', goal)
else:
    print('[INFO] 正規化不要')

# ★検証: スキーマ整合性。失敗したら上流(プロンプト/正規化)を調整
plan = Plan.model_validate(data)
print('[VALIDATION] OK main_task=', plan.main_task, ' subtasks=', len(plan.subtasks))

# ★可観測性: 部分プレビューで異常値(極端な長さ/重複)を早期発見
for st in plan.subtasks[:6]:
    print(' -', st.assigned_agent, '|', st.task_details)
if len(plan.subtasks) > 6:
    print(' ...(他', len(plan.subtasks)-6, '件)')

print('\n[GUIDE] goal を変更 / 温度変更 / SystemPrompt 強化 / 正規化ロジック改良 で品質比較')

[RAW 頭120] {"main_task":"シンガポールからメルボルンへの3日旅行プラン","subtasks":[{"task_details":"往復航空券を検索","assigned_agent":"flight_booking"},{"task_d ...
[PARSE] 抽出候補 先頭90: {"main_task":"シンガポールからメルボルンへの3日旅行プラン","subtasks":[{"task_details":"往復航空券を検索","assigned_age ...
[INFO] 正規化不要
[VALIDATION] OK main_task= シンガポールからメルボルンへの3日旅行プラン  subtasks= 5
 - AgentEnum.Flight | 往復航空券を検索
 - AgentEnum.Hotel | メルボルンでの宿泊先を予約
 - AgentEnum.Car | メルボルンでのレンタカーを手配
 - AgentEnum.Activity | メルボルンでの観光アクティビティを計画
 - AgentEnum.Destination | メルボルンの観光情報を提供

[GUIDE] goal を変更 / 温度変更 / SystemPrompt 追記で再実験


## セル7: 拡張案 (次ステップの指針)
以下は学習深化のための発展項目です。

1. function calling で各 subtask を自動実行
2. プラン品質メトリクス: ステップ数・重複・カテゴリ網羅
3. Re-plan: 失敗/追加要件時に差分再生成 (前回 Plan をコンテキスト投入)
4. マルチエージェント: router + executor + aggregator 分離
5. ロギング/評価: JSON ログ保存 & 再現テスト (回帰検知)
6. diff 表示: 前回と今回の subtasks 差分強調
7. 評価用ベンチ: 複数ゴールを一括検証 (成功率/平均ステップ)

> 推奨: Re-plan / Multi-agent を別ノート `replan.ipynb` `multi_agent.ipynb` に分け、最小例との責務境界を維持。